Load API Keys

In [11]:
from dotenv import load_dotenv
load_dotenv()

True

Load Contents from Brillar Bank

In [12]:
from langchain_community.document_loaders import WebBaseLoader
from helpers.test_data import get_urls

loader = WebBaseLoader(get_urls())

data = loader.load()

Split the Loaded Data into Chunks

In [13]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=20)
texts = text_splitter.split_documents(data)

Get Embedding Model

In [14]:
from langchain_openai import OpenAIEmbeddings

embedding = OpenAIEmbeddings(model="text-embedding-3-large", dimensions=256)

Instantiate Elasticsearch Store

In [15]:
from langchain_elasticsearch import ElasticsearchStore, BM25Strategy, DenseVectorStrategy
import os

es_cloud_id = os.getenv("ELASTIC_CLOUD_ID")
es_api_key = os.getenv("ELASTIC_API_KEY")

dense_vector_store = ElasticsearchStore.from_documents(
    documents=texts,
    es_cloud_id=es_cloud_id,
    es_api_key=es_api_key,
    index_name="bm25_dense",
    embedding=embedding,
    strategy=DenseVectorStrategy()
)

bm25_store = ElasticsearchStore.from_documents(
  documents=texts,
  es_cloud_id=es_cloud_id,
  es_api_key=es_api_key,
  index_name="bm25",
  strategy=BM25Strategy()
)

Get LLM

In [16]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o")

Get Retriever

In [17]:
from langchain.retrievers import EnsembleRetriever

dense_retriever = dense_vector_store.as_retriever(search_kwargs={"k": 5})
bm25_retriever = bm25_store.as_retriever(search_kwargs={"k": 5})

retriever = EnsembleRetriever(retrievers=[dense_retriever, bm25_retriever])

Create Chain

In [18]:
from helpers.conversation_retrieval_chain import create_conversational_retrieval_chain

chain = create_conversational_retrieval_chain(llm, retriever)

Invoke Chain

In [19]:
from helpers.conversation_retrieval_chain import invoke_conversational_retrieval_chain
from helpers.test_data import get_questions

questions = get_questions()
for question in questions:
  result = invoke_conversational_retrieval_chain(chain, question)
  print(f'{question}\n{result["answer"]}\n')

How many types of fixed deposit does Hong Leong Bank provide?
Hong Leong Bank provides six types of fixed deposit accounts: Fixed Deposit, e-Fixed Deposit, Flexi Fixed Deposit, Senior Savers Flexi Fixed Deposit, Junior Fixed Deposit, and Foreign Currency Fixed Deposit.

What are the interest rates for fixed deposit?
The interest rates for fixed deposits are as follows:

- 1 month: 2.15% p.a.
- 2-3 months: 2.25% p.a.
- 4-5 months: 2.30% p.a.
- 6 months: 2.30% p.a.
- 7-11 months: 2.35% p.a.
- 12-60 months: 2.50% p.a.

What are the interest rates for e-fixed deposit?
The interest rates for Hong Leong Bank's e-fixed deposit are as follows:

- 1 month: 1.85% p.a.
- 2-3 months: 2.25% p.a.
- 4-6 months: 2.30% p.a.
- 7-12 months: 2.35% p.a.
- 13-60 months: 2.35% p.a.

What are the interest rates for flexi-fixed deposit?
The interest rates for the Hong Leong Bank Flexi Fixed Deposit are as follows:

- 1 month: 2.15% p.a.
- 2 - 3 months: 2.25% p.a.
- 4 - 5 months: 2.30% p.a.
- 6 months: 2.30% p.